## 6.2 Entraînement du modèle DELTA_DIST+H_01 - Entraînement avec RMSE total

Nous allons ici modifier la fonction de coût lors de l'entraînement, dans l'idée que cela aidera peut-être le modèle à faire de meilleures prédictions.

Nous allons pour cela utiliser une fonction calculant le RMSE total des prédictions, c'est à dire sans appliquer de masque sélectionnant uniquement les prédictions correspondant aux atomes définis en entrée du réseau. L'optimiseur va donc avoir plus de souplesse et pourra peut être converger vers des solutions plus intéressantes.

Cependant, nous utiliserons toujours le RMSE partiel comme mesure de validation du modèle, à la place du score R2 comme c'est le cas jusque ici.


#### Résultats

Cette méthode est finalement moins efficace que les méthodes précédentes. La loss totale est plus faible (car il est plus facile pour le modèle de prédire des zéros pour les valeurs des atomes non définis), mais le RMSE partiel stagne au niveau du RMSE introduit par le bruit (170). Il semblerait donc que l'utilisation du RMSE partiel comme loss guide plus efficacement le modèle dans la bonne direction.

#### Chemin des fichiers

In [1]:
minimal_prepared_input_loc = "../data/minimal_set_riken_v2_prepared_input_bruit+.h5"
minimal_labels_loc = "../data/minimal_set_riken_v2_labels_bruit+.h5"

train_prepared_input_loc = "../data/train_set_riken_v2_prepared_input_bruit+.h5"
train_labels_loc = "../data/train_set_riken_v2_labels_bruit+.h5"

mini_prepared_input_loc = "../data/mini_set_prepared_input_bruit+.h5"
mini_labels_loc = "../data/mini_set_labels_bruit+.h5"


models_loc = "../models/DELTA_DIST+H_01/6.2.RMSE_TOTAL/"
logs_loc = "../models/DELTA_DIST+H_01/6.2.RMSE_TOTAL/"

## Création du RN

### Fonctions de coût et d'évaluation du modèle

Les fonctions ci-dessous sont très fortement inspirées du travail de Nicolas Roux lors de son TER de M1 en 2017. Les différences sont les suivantes.

* Calcul du RMSE uniquement sur les atomes définis
* Utilisation d'un score R2 pour la validation
* Recherche par quadrillage des hyper-paramètres

#### Fonction renvoyant le masque des atomes à prendre en compte pour les calculs

L'entrée et la sortie du RN étant définies par une méthode de padding, seul un certain nombre d'entrées et de sortie est utilisé pour chaque exemple d'entraînement en fonction du nombre d'atomes de la molécule. On définit ici une fonction qui renvoie le masque des différences de distances à prendre en compte sur les données en entrée et les étiquettes.

In [14]:
import tensorflow as tf

def calcul_masque_atomes_definis(targets):
    """ On calcule le masque booléen des atomes donnés en entrée du RN en fonction du vecteur targets"""
    
    # On cherche à obtenir un masque booléen des atomes définis en entrée. Pour cela, on prend en entrée
    # les étiquettes sous la forme d'une matrice (200, 4) dont chaque ligne i est la distance de l'atome i avec
    # les atomes fictifs du repère. L'atome est indéfini ssi. la somme de la ligne est nulle. En effet,
    # un atome défini ne peut pas avoir une distance nulle avec les quatre atomes fictifs, et on veille
    # à ce que le vecteurs targets ne contienne que des valeurs nulles pour les atomes non définis.
    # On obtient donc un masque booléen de tous les atomes définis en entrée
    
    ## On somme les distances de chaque atome ##
    targets_dists_sums = tf.reduce_sum(targets, 1)
    
    ## On créé le masque des sommes différentes de zéro ##
    
    # Création des matrice de True et de False de la dimension de la matrice des sommes (nécessaires
    # pour tf.where)
    zeros = tf.cast(tf.zeros_like(targets_dists_sums),dtype=tf.bool)
    ones = tf.cast(tf.ones_like(targets_dists_sums),dtype=tf.bool)
    
    return tf.where(targets_dists_sums>0, ones, zeros)


#### Fonction de coût partiel

In [15]:
def partial_rmse(predictions, targets, inputs):
    """ Calcule le RMSE partiel des prédictions par rapport aux valeurs attendues. Le RMSE est partiel car
    on ne le calcule que pour les sorties correspondant aux atomes donnés en entrée. En d'autres
    termes, on ne pousse pas le modèle à donner des distances nulles pour les atomes indéfinis
    en entrée"""
    
    with tf.name_scope("partial_rmse"):

        # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
        predictions = tf.reshape(predictions, [-1, 4])
        targets = tf.reshape(targets, [-1, 4])

        # On calcule le masque des atomes définis selon les cibles
        defined_atoms_mask = calcul_masque_atomes_definis(targets)
        
        # On masque les prédictions et les étiquettes selon le masque des atomes définis
        targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
        predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)   

        return tf.sqrt(tf.reduce_mean(tf.squared_difference(predictions_masked, targets_masked)), 
                       name="partial_rmse")

#### Fonction de coût total (sans masque)

In [16]:
def rmse(predictions, targets):
    with tf.name_scope("total_rmse"):
        return tf.sqrt(tf.reduce_mean(tf.squared_difference(predictions, targets)), name="rmse")
    
    

#### Fonction d'évaluation des performances (score R2)

In [17]:
def partial_r2_score(predictions, targets, inputs):
    """ Renvoie le score R2 de la prédiction (le calcul est effectué uniquement sur les résultats
    des atomes donnés en entrée) """
    
    with tf.name_scope("partial_r2"):
    
        # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
        predictions = tf.reshape(predictions, [-1, 4])
        targets = tf.reshape(targets, [-1, 4])

        # On calcule le masque des atomes définis selon les cibles
        defined_atoms_mask = calcul_masque_atomes_definis(targets)

        # On masque les prédictions et les étiquettes selon le masque des atomes définis
        targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
        predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)

        # Calcul de l'erreur totale
        total_error = tf.reduce_sum(tf.square(tf.subtract(targets, tf.reduce_mean(targets_masked))))

        # Calcul de l'erreur inexpliquée
        unexplained_error = tf.reduce_sum(tf.square(tf.subtract(targets_masked, predictions_masked)))

        r2 = tf.subtract(1.0, tf.divide(unexplained_error, total_error), "r2_score")
        return r2

#### Définition d'une fonction créant le RN

In [18]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.optimizers import Adam
from tflearn.data_preprocessing import DataPreprocessing
import math


def creer_1k_3x8k_800(epsilon=1e-8, learning_rate=0.001, dropout_val=0.99, stddev_init=0.001,
                      hidden_act='relu', outlayer_act='prelu'):
    """ Fonction créant un réseau de neurones de type fully connected, ayant une couche d'entrée de 1000
    neurones, quatre couches cachées de 8650 neurones et une sortie de 800 neurones
    Inputs : hyperparamètres
    """

    # On créé l'initialisateur de tenseur avec une loi normale tronquée. sigma = stddev_init, et les 
    # valeurs à plus de 2sigma sont re-tirées
    winit = tfl.initializations.truncated_normal(stddev=stddev_init, dtype=tf.float32, seed=None)
    
    # On créé l'input du RN
    network = input_data(shape=[None, 1000], name='input')
    
    # crelu ayant besoin de tenseurs deux fois plus importants, on adapte la taille du modèle
    if hidden_act == 'crelu':
        depth = 2
        width = 8000
    else:
        depth = 3
        width = 8000
    
    # On créé les couches cachées
    for i in range(depth):
        network = fully_connected(network, width, activation=hidden_act, name='fc'+str(i), weights_init=winit)
        # On détruit des neurones aléatoirement avec une la probabilité donnée en entrée
        network = dropout(network, dropout_val)
    
    # On ajoute la couche de sortie du réseau
    # Fonction d'activation prelu
    # Initilisée avec la loi normale tronquée
    network = fully_connected(network, 800, activation=outlayer_act, name='outlayer', weights_init=winit)
    
    adam = Adam(learning_rate=learning_rate, epsilon=epsilon)
    
    # Couche d'évaluation du modèle. Utilisation d'une descente stochastique Adam
    # Learning rate = 0.05
    # Loss = fonction définie rmse
    network = regression(network, optimizer=adam,
    loss=rmse, metric=partial_rmse, name='target')
            
    return network

## Préparation des données


#### Fonction renvoyant deux sous-ensembles du jeu d'entrainement : un ensemble d'exemples et les cibles correspondantes

In [19]:
def get_fold(train_set, targets, reduce_train_fold_size):
    """ Permet d'obtenir un sous-ensemble du jeu d'entraînement afin de ne pas travailler sur le jeu
    d'entraînement total pour la recherche par quadrillage et donc de gagner du temps d'exécution. L'idée
    et que si un ensemble d'hyper-paramètres produit des meilleurs résultats que les autres ensembles
    d'hyper-paramètres sur l'ensemble du jeu d'entraînement, alors on suppose que ce sera également 
    le cas sur une partie des données. """

    return (train_set["inputs"][:reduce_train_fold_size], targets["targets"][:reduce_train_fold_size])
    

## Entraînement des modèles

#### Fonction d'entraînement d'un modèle

In [20]:
import h5py
import tflearn as tfl
import time
from scipy import sparse
import numpy as np
import gc
import tensorflow as tf


def train_model(input_X, labels_y, model_name, model_path, logs_path, samples_per_batch=1000, epochs=5,
                learning_rate=0.001, epsilon=1e-8, dropout=0.99, stddev_init=0.001, hidden_act='relu',
                outlayer_act='prelu'):
    
    total_start_time = time.time()
    
    tf.reset_default_graph()

    # On créé le réseau 
    network = creer_1k_3x8k_800(learning_rate=learning_rate, epsilon=epsilon, dropout_val=dropout,
                                 stddev_init=stddev_init, hidden_act=hidden_act, outlayer_act=outlayer_act)

    # On créé le modèle
    model = tfl.DNN(network, tensorboard_verbose=3, tensorboard_dir=logs_path)

    # Entraînement
    model.fit(X_inputs=input_X,Y_targets=labels_y, batch_size=samples_per_batch,
              shuffle = True, snapshot_step=100, validation_set=0.1,
              show_metric=True, run_id=model_name , n_epoch=epochs)

    # Sauvegarde du modèle
    #model.save(model_path + model_name)


In [21]:
import h5py


input_X_h5 = h5py.File(train_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(train_labels_loc, 'r')

"""
input_X_h5 = h5py.File(mini_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(mini_labels_loc, 'r')
"""
input_X, labels_y = get_fold(input_X_h5, labels_y_h5, 3000000)

model_name = "DELTA_DIST+H_01_6.2"
model_path = models_loc
logs_path = logs_loc

train_model(input_X, labels_y, model_name, model_path, logs_path, samples_per_batch=1000, 
            epochs=1, learning_rate=0.01, dropout=0.95, epsilon=0.001, hidden_act="relu6",
            outlayer_act="linear")


ResourceExhaustedError: OOM when allocating tensor of shape [10000,10000] and type float
	 [[Node: fc1/W/Adam/Initializer/zeros = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [10000,10000] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'fc1/W/Adam/Initializer/zeros', defined at:
  File "/home/etudiant/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/etudiant/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/etudiant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/etudiant/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/etudiant/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/etudiant/anaconda3